In [4]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")


gams_python_path="/Library/Frameworks/GAMS.framework/Resources/apifiles/Python/gams"

#### Import Data

In [5]:
model=pd.read_csv('/Users/hannahkamen/Downloads/windc_ind.csv')
cps=pd.read_stata('/Users/hannahkamen/Downloads/acs_20132018_5yr.dta')

lookup=pd.read_excel('/Users/hannahkamen/Downloads/windc_cps_industry_lookup.xlsx')
lookup_final=pd.read_excel('/Users/hannahkamen/Downloads/dropped_merge.xlsx')
state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/state_lookup.xlsx')

lookup['NAICS']=lookup['NAICS'].astype(str).str.strip()
lookup_final['NAICS']=lookup_final['NAICS'].astype(str).str.strip()
lookup_final['cps_code']=lookup_final['cps_code'].astype(str).str.strip()

#cps = np.loadtxt('/Users/hannahkamen/Downloads/usa_00002.dat', unpack = True)



In [6]:
len(cps)

93921408

In [8]:
state_lookup['state']=state_lookup['state'].str.strip()
state_lookup['abbrev']=state_lookup['abbrev'].str.strip()

In [9]:
###read in dropped 

In [10]:
cps_codes=pd.DataFrame(data={'cps_code':cps['indnaics'].str.strip().unique()})
windc_codes=pd.DataFrame(data={'windc_code':model['IOCode'].str.strip().unique()})

In [11]:
####match on first three symbols
cps_codes['cps_code3']=cps_codes['cps_code'].str[0:3]
lookup['NAICS3']=lookup['NAICS'].str[0:3]

merge3=lookup.merge(cps_codes, left_on='NAICS3',right_on='cps_code3',how='inner',indicator=True)

In [12]:
###append manual lookup to fill gaps
merge_f=merge3[['NAICS','cps_code']].append(lookup_final)

In [13]:
###get state symbol for place of residence
cps['statefip']=cps['statefip'].str.lower().str.strip()
cps['pwstate2']=cps['pwstate2'].str.lower().str.strip()

cps_m=cps.merge(merge_f, left_on='indnaics', right_on='cps_code',how='left').merge(state_lookup, left_on='statefip',right_on='state', how='left')
cps_m=cps_m.rename(columns={'abbrev':'state_residence'})

###get state symbol for place of work
cps_m=cps_m.merge(state_lookup, left_on='pwstate2', right_on='state',how='left')
cps_m=cps_m.rename(columns={'abbrev':'state_work'})

### get skilled unskilled breakouts
cps_m['educ_new']=np.where(cps_m['educ'].isin(['2 years of college','4 years of college','5+ years of college']),'skl','unskl')
 


In [14]:
###prep household variable
cps_m['hh']=np.where(cps_m['inctot']<=25000,'hh1','')
cps_m['hh']=np.where(((cps_m['inctot']>25000) & (cps_m['inctot']<50000)),'hh2',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['inctot']>=50000) & (cps_m['inctot']<75000)),'hh3',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['inctot']>=75000) & (cps_m['inctot']<150000)),'hh4',cps_m['hh'])
cps_m['hh']=np.where((cps_m['inctot']>=150000),'hh5',cps_m['hh'])

In [15]:
non_s=['Output',
       'imports', 'ciffob', 'BasicSupply', 'Margins', 'TrnCost', 'TrdTrn',
       'Duties', 'Tax', 'Subsidies', 'TaxLesSubsidies', 'Supply']

states=cps_m['state_residence'].unique()

In [16]:
####limit to workers with location assigned for work and residence
cps_m1=cps_m[(cps_m['state_work'].isin(states))& (cps_m['state_residence'].isin(states))]


In [17]:
len(cps_m1)

54341566

### Assign Sector Values and Limit Data

In [18]:
#list of US states

#### drop if unemployed
#cps_m1=cps_m[(cps_m['indnaics']!='0') & (cps_m['indnaics']!='') & (cps_m['state_work'].isin(states))]

#### group data by all relevant breakous - r,q,h,s,sk
all_breakouts0=cps_m1.reset_index()[['state_residence','state_work','NAICS','hh','educ_new','incwage']].groupby(['state_residence','state_work','NAICS','hh','educ_new'],as_index=False).agg({'incwage':sum})

#rename variables and limit dataframe to relevant sectors
all_breakouts0=all_breakouts0.rename(columns={'state_residence':'r','state_work':'q','NAICS':'s','hh':'h','educ_new':'sk'})
all_breakouts0=all_breakouts0[['r','q','s','h','sk','incwage']]

### now group model dataset by relevant Naics codes and Windc specific codes
model_gr=model.groupby(['IOCode','gams.IOCode'],as_index=False).sum()
model_gr=model_gr[['IOCode','gams.IOCode']]

###merge with cps data 
all_breakouts0=all_breakouts0.merge(model_gr, left_on='s',right_on='IOCode',how='left')

##get rid of used sector

all_breakouts0=all_breakouts0[all_breakouts0['gams.IOCode']!='use']

##now delete old NAICS var and re-assign ind var as the values in 'gams.IOCode'
del all_breakouts0['s']
del all_breakouts0['IOCode']
all_breakouts0=all_breakouts0.rename(columns={'gams.IOCode':'s'})
all_breakouts=all_breakouts0[['r','q','s','h','sk','incwage']]



In [67]:
all_breakouts.head()

,r,q,s,h,sk,incwage,census_region_r,census_region_q
0,AK,AK,agr,hh1,skl,262789.0,west,west
1,AK,AK,agr,hh1,unskl,816808.0,west,west
2,AK,AK,agr,hh2,skl,423309.0,west,west
3,AK,AK,agr,hh2,unskl,151668.0,west,west
4,AK,AK,agr,hh3,skl,1222716.0,west,west


#### get current model breakouts region lived in,region worked, household type le0(r,q,h)

In [70]:
###classify census regions
midwest=['IA','OH','WI','NE','IL','MI','SD','ND','MN','MO','IN','KS']
south=['FL','MD','TN','WV','OK','KY','NC','VA','DE','GA','MS','TX','AL','LA','AR','SC','DC']
west=['AK','AZ','NM','HI','CA','WA','NV','OR','ID','UT','MT','WY','CO']
northeast=['VT','NH','CT','ME','MA','NY','NJ','PA','RI']

###get census region lived in, census region worked in

all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(midwest),'midwest','')
all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(south),'south',all_breakouts['census_region_r'])
all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(west),'west',all_breakouts['census_region_r'])
all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(northeast),'northeast',all_breakouts['census_region_r'])


all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(midwest),'midwest','')
all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(south),'south',all_breakouts['census_region_q'])
all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(west),'west',all_breakouts['census_region_q'])
all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(northeast),'northeast',all_breakouts['census_region_q'])

####limit to where person works in same census region that they live in
all_breakouts_lm=all_breakouts[(all_breakouts['census_region_r']==all_breakouts['census_region_q'])]

###get total income by r,q then get income by r,q,h calculate to get share household income by income type
#total by rqh
all_breakouts_lm_gr=all_breakouts_lm.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})

##total by r,q
# all_breakouts_lm_tot=all_breakouts_lm.groupby(['r','q'],as_index=False).agg({'incwage':sum})

# all_breakouts_lm_tot=all_breakouts_lm_tot.rename(columns={'incwage':'incwage_total'})

# le0_breakouts=all_breakouts_lm_gr.merge(all_breakouts_lm_tot, on=['r','q'],how='left')

# le0_breakouts['hh_shr']=le0_breakouts['incwage']/le0_breakouts['incwage_total']
le0_breakouts=all_breakouts_lm.groupby(['census_region_r','h'],as_index=False).agg({'incwage':sum})


###get total income by r,s then get income by r calculate to get share of labor expenditures in each regionfor each sector

all_breakouts_lm_gr1=all_breakouts_lm.groupby(['r','s'],as_index=False).agg({'incwage':sum})

##total by r,q
# all_breakouts_lm_tot1=all_breakouts_lm.groupby(['r'],as_index=False).agg({'incwage':sum})

# all_breakouts_lm_tot1=all_breakouts_lm_tot1.rename(columns={'incwage':'incwage_total'})

# ld0_breakouts=all_breakouts_lm_gr1.merge(all_breakouts_lm_tot1, on=['r'],how='left')

# ld0_breakouts['r_shr']=ld0_breakouts['incwage']/ld0_breakouts['incwage_total']
ld0_breakouts=all_breakouts_lm.groupby(['census_region_q','s'],as_index=False).agg({'incwage':sum})



/var/folders/w1/12ls607j77q1jk587t2n5t2r0000gn/T/ipykernel_29526/744693783.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(midwest),'midwest','')
/var/folders/w1/12ls607j77q1jk587t2n5t2r0000gn/T/ipykernel_29526/744693783.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(south),'south',all_breakouts['census_region_r'])
/var/folders/w1/12ls607j77q1jk587t2n5t2r0000gn/T/ipyk

#### Get missing combinations for both datasets

In [180]:
####gams labor demand data
gams_dta_ld0=pd.read_csv('/Users/hannahkamen/Downloads/windc_ld0_d2.csv')

####gams labor endowment data
gams_dta_le0=pd.read_csv('/Users/hannahkamen/Downloads/windc_le0_d.csv')

In [240]:
###group labor endowment by place of work and sector. These are all the place of work and sector combinations
### in the model that we need shares for

gams_dta_ld0=gams_dta_ld0.rename(columns={' flx':'q',' int':'s',' scn':'sk',' esubw_oth':'pop'})

gams_dta_ld0['census_region_q']=np.where(gams_dta_ld0['q'].isin(midwest),'midwest','')
gams_dta_ld0['census_region_q']=np.where(gams_dta_ld0['q'].isin(south),'south',gams_dta_ld0['census_region_q'])
gams_dta_ld0['census_region_q']=np.where(gams_dta_ld0['q'].isin(west),'west',gams_dta_ld0['census_region_q'])
gams_dta_ld0['census_region_q']=np.where(gams_dta_ld0['q'].isin(northeast),'northeast',gams_dta_ld0['census_region_q'])

ld00=gams_dta_ld0[['q','s','sk','pop','census_region_q']]
ld0=gams_dta_ld0.rename(columns={' flx':'q',' int':'s',' esubw_oth':'pop'})[['census_region_q','s','pop']].groupby(['census_region_q','s'],as_index=False).sum()

##prepare le0 table

gams_dta_le0=gams_dta_le0.rename(columns={' flx':'r',' int':'q',' scn':'h',' esubw_oth':'sk',' sector':'pop'})

gams_dta_le0['census_region_r']=np.where(gams_dta_le0['r'].isin(midwest),'midwest','')
gams_dta_le0['census_region_r']=np.where(gams_dta_le0['r'].isin(south),'south',gams_dta_le0['census_region_r'])
gams_dta_le0['census_region_r']=np.where(gams_dta_le0['r'].isin(west),'west',gams_dta_le0['census_region_r'])
gams_dta_le0['census_region_r']=np.where(gams_dta_le0['r'].isin(northeast),'northeast',gams_dta_le0['census_region_r'])

gams_dta_le0['census_region_q']=np.where(gams_dta_le0['q'].isin(midwest),'midwest','')
gams_dta_le0['census_region_q']=np.where(gams_dta_le0['q'].isin(south),'south',gams_dta_le0['census_region_r'])
gams_dta_le0['census_region_q']=np.where(gams_dta_le0['q'].isin(west),'west',gams_dta_le0['census_region_r'])
gams_dta_le0['census_region_q']=np.where(gams_dta_le0['q'].isin(northeast),'northeast',gams_dta_le0['census_region_r'])


le00=gams_dta_le0.rename(columns={' flx':'r',' int':'q',' scn':'h',' sector':'pop'})[['r','q','sk','census_region_r','h','pop','census_region_q']]
le0=le00.groupby(['census_region_r','h'],as_index=False).sum()
## merge region lived in and region worked in combinations with all sector combinations for the region worked in (q)

le0_matches=le0.merge(le0_breakouts,on=['census_region_r','h'],how='outer',indicator=True)
le0_matches=le0_matches[le0_matches['_merge']!='both']
le0_matches=le0_matches[['census_region_r','h','pop','incwage','_merge']]

ld0_matches=ld0.merge(ld0_breakouts,on=['census_region_q','s'],how='outer',indicator=True)
ld0_matches=ld0_matches[ld0_matches['_merge']!='both']
ld0_matches=ld0_matches[['census_region_q','s','pop','incwage','_merge']]




In [371]:
###get share of total labor in each census region by skill

tot_r=all_breakouts_lm.groupby(['census_region_q'],as_index=False).agg({'incwage':sum})
tot_r=tot_r.rename(columns={'incwage':'incwage_tot'})
tot_r_sk=all_breakouts_lm.groupby(['census_region_q','sk'],as_index=False).agg({'incwage':sum})

tot_r_sk=tot_r_sk.merge(tot_r,on=['census_region_q'],how='inner')
tot_r_sk['share_labor_region']=tot_r_sk['incwage']/tot_r_sk['incwage_tot']
del tot_r_sk['incwage']
del tot_r_sk['incwage_tot']

##get total labor endowment by region and skill in gams

le00_st=le00.groupby(['census_region_q'],as_index=False).sum()

tot_r_sk=tot_r_sk.merge(le00_st, on=['census_region_q'],how='right')

tot_r_sk['pop_skill_region']=tot_r_sk['share_labor_region']*tot_r_sk['pop']

##this dataset has the total labor endowment by skill in each region using the GAMS table populations by region:

tot_r_sk=tot_r_sk[['census_region_q','sk','pop_skill_region']]

In [534]:
0.619120*1771.068605

1096.5039947276

In [533]:
tot_r_sk

,census_region_q,sk,share_labor_region,pop,pop_skill_region
0,midwest,skl,0.619120,1771.068605,1096.504320
1,midwest,unskl,0.380880,1771.068605,674.564285
2,northeast,skl,0.712862,1798.783661,1282.285148
3,northeast,unskl,0.287138,1798.783661,516.498513
4,south,skl,0.649778,3307.911132,2149.409358
5,south,unskl,0.350222,3307.911132,1158.501774
6,west,skl,0.672765,2337.991157,1572.917455
7,west,unskl,0.327235,2337.991157,765.073702


In [532]:
tot_r_sk['pop_skill_region'].sum()

9215.754554875695

In [550]:
tot_r_sk

,census_region_q,sk,share_labor_region,pop,pop_skill_region
0,midwest,skl,0.619120,1771.068605,1096.504320
1,midwest,unskl,0.380880,1771.068605,674.564285
2,northeast,skl,0.712862,1798.783661,1282.285148
3,northeast,unskl,0.287138,1798.783661,516.498513
4,south,skl,0.649778,3307.911132,2149.409358
5,south,unskl,0.350222,3307.911132,1158.501774
6,west,skl,0.672765,2337.991157,1572.917455
7,west,unskl,0.327235,2337.991157,765.073702


In [475]:
28.004075/1771.068605

0.01581196511582904

In [476]:
le00_rh

,census_region_q,h,pop,total_r_pop,regional_hh_share
0,midwest,hh1,28.004075,1771.068605,0.015812
1,midwest,hh2,132.272487,1771.068605,0.074685
2,midwest,hh3,215.728771,1771.068605,0.121807
3,midwest,hh4,655.735493,1771.068605,0.370248
4,midwest,hh5,739.327780,1771.068605,0.417447
5,northeast,hh1,19.485658,1798.783661,0.010833
6,northeast,hh2,101.290428,1798.783661,0.056311
7,northeast,hh3,161.442840,1798.783661,0.089751
8,northeast,hh4,549.368593,1798.783661,0.305411
9,northeast,hh5,967.196141,1798.783661,0.537695


##### get share of total skilled endowment in each region allocated to each household
##### this datset shows the percent of total skill type that each household is endowed with in each region

##### need to calibrate to actual householde population levels in gams data
1) get household populations in each region in gams data
2) multiply household populations by percent of skill type that each is endowed with
3) calculate breakouts this way

In [547]:
#######get breakout of household by region in gams data

#get shares of total region population that each hopusehold comprises
le00_r=le00.groupby(['census_region_q'],as_index=False).sum()
le00_r=le00_r.rename(columns={'pop':'total_r_pop'})
le00_rh=le00.groupby(['census_region_q','h'],as_index=False).sum()
le00_rh=le00_rh.merge(le00_r,on='census_region_q')
le00_rh['regional_hh_share']=le00_rh['pop']/le00_rh['total_r_pop']
le00_rh=le00_rh[['census_region_q','h','pop']]

##the above dataset has the regional population by household as specified in GAMS data
### now get percent of total labor type endowed to each household in wage data
####get these values that do not reflect actual amounts in gams data
#####calculate shares from these values and apply to actual labor endowment by hh type in le00_rh


#######tot_r_sk_h_raw gives us the percentage of total labor endowment by skill type that each household
#########is endowed with in each region

tot_r_sk_raw=all_breakouts_lm.groupby(['census_region_q','sk'],as_index=False).agg({'incwage':sum})
tot_r_sk_raw=tot_r_sk_raw.rename(columns={'incwage':'incwage_tot'})
tot_r_sk_h_raw=all_breakouts_lm.groupby(['census_region_q','sk','h'],as_index=False).agg({'incwage':sum})

tot_r_sk_h_raw=tot_r_sk_h_raw.merge(tot_r_sk_raw,on=['census_region_q','sk'],how='inner')
tot_r_sk_h_raw['share_hh_bysk']=tot_r_sk_h_raw['incwage']/tot_r_sk_h_raw['incwage_tot']
del tot_r_sk_h_raw['incwage']
del tot_r_sk_h_raw['incwage_tot']

tot_r_sk_h_raw=tot_r_sk_h_raw[['sk','census_region_q','h','share_hh_bysk']]
#....HH1 GETS 1% OF SKILLED LABOR IN MIDWEST AND SO ON.....
#NOW NEED SHARE OF TOTAL LABOR THAT SKILLED AND UNSKILLED IS IN EACH REGION. SET THIS RATIO IN GAMS DATA
# tot_r_sk TELLS US TOTAL LABOR IN EACH REGION by EACH SKILL TYPE AND SUMS TO TOTAL LABOR IN GAMS

#SO USING tot_r_sk_h_raw WE CAN SAY HH1 GETS 1% OF SKILLED LABOR IN REGION WHICH IS 1% of 61.91% OF THE TOTAL POP

#ASSIGN THESE LVEL VALUES TO HH1, THEN RECALCULATE SHARES


###HH1 is 10% of total labor demand in region in gams, 
### HH1 demands 1% total unskilled and 2% of total skilled labor in the region and 10% of total labor
###this woould put them at 50 demanded but gams says they only get 28 units endowed
### HH1 can only be 28 percent of total demand in gams

#...okay so need to calibrate to industry size in GAMS what is 10% of total labor in region, 28 units 

####merge on total labor endowment by region 

tot_r_sk_lm=tot_r_sk[['census_region_q','sk','pop_skill_region']]

get_hh_splits=tot_r_sk_h_raw.merge(tot_r_sk_lm,on=['census_region_q','sk'])

###multiply the percent of total labor in region by skill each household gets by total endow of labor type in that region
get_hh_splits['hh_endow_bysk']=get_hh_splits['share_hh_bysk']*get_hh_splits['pop_skill_region']

tot_hh_endow=get_hh_splits.groupby(['census_region_q','h'],as_index=False).agg({'hh_endow_bysk':sum})

tot_hh_endow=tot_hh_endow.rename(columns={'hh_endow_bysk':'tot_hh_endowment'})
tot_hh_endow=tot_hh_endow[['census_region_q','h','tot_hh_endowment']]
get_hh_splits=get_hh_splits.merge(tot_hh_endow,on=['census_region_q','h'])

#### get expenditure share by skill type 
get_hh_splits['expenditure_share']=get_hh_splits['hh_endow_bysk']/get_hh_splits['tot_hh_endowment']
get_hh_splits=get_hh_splits[['sk','census_region_q','h','expenditure_share']]
####get actual household labor endowment from gams

cons_final=get_hh_splits.merge(le00_rh,on=['census_region_q','h'])
cons_final['pop_hh_sk']=cons_final['expenditure_share']*cons_final['pop']
# ####merge share skill type by household onto previous table expressing share of labor type in each region

# cons_final=tot_r_sk_h_raw.merge(tot_r_sk,on=['census_region_q','sk'])
# cons_final['pop_region_skill_hh']=cons_final['share_hh_bysk']*cons_final['pop_skill_region']
# cons_final=cons_final[['census_region_q','h','sk','pop_region_skill_hh','share_hh_bysk']]


In [556]:
tot_r_sk_cal=cons_final.groupby(['sk','census_region_q'],as_index=False).agg({'pop_hh_sk':sum})
tot_r_sk_cal=tot_r_sk_cal.rename(columns={'pop_hh_sk':'pop_skill_region'})

In [1]:
cons_final.head()

NameError: name 'cons_final' is not defined

In [557]:
tot_r_sk_cal


,sk,census_region_q,pop_skill_region
0,skl,midwest,1323.321559
1,skl,northeast,1458.065260
2,skl,south,2479.154734
3,skl,west,1808.007940
4,unskl,midwest,447.747046
5,unskl,northeast,340.718401
6,unskl,south,828.756398
7,unskl,west,529.983217


##### get share of total skilled endowment in each region allocated to each sector

In [566]:
#######get breakout of sector by region in gams data

#get shares of total region population that each hopusehold comprises
ld00_r=ld00.groupby(['census_region_q'],as_index=False).sum()
ld00_r=ld00_r.rename(columns={'pop':'total_r_pop'})
ld00_rh=ld00.groupby(['census_region_q','s'],as_index=False).sum()
ld00_rh=ld00_rh.merge(ld00_r,on='census_region_q')
ld00_rh['regional_s_share']=ld00_rh['pop']/ld00_rh['total_r_pop']
ld00_rh=ld00_rh[['census_region_q','s','pop']]

##the above dataset has the regional demand by sector as specified in GAMS data
### now get percent of total demand by sector in wage data by skill type
####get these values of demand that do not reflect actual amounts in gams data
#####calculate shares from these values and apply to actual labor endowment by s type in ld00_rh


#######tot_r_sk_s_raw gives us the percentage of total sector demand by skill typein each region

tot_r_sk_raw=all_breakouts_lm.groupby(['census_region_q','sk'],as_index=False).agg({'incwage':sum})
tot_r_sk_raw=tot_r_sk_raw.rename(columns={'incwage':'incwage_tot'})
tot_r_sk_s_raw=all_breakouts_lm.groupby(['census_region_q','sk','s'],as_index=False).agg({'incwage':sum})

tot_r_sk_s_raw=tot_r_sk_s_raw.merge(tot_r_sk_raw,on=['census_region_q','sk'],how='inner')
tot_r_sk_s_raw['share_s_bysk']=tot_r_sk_s_raw['incwage']/tot_r_sk_s_raw['incwage_tot']
del tot_r_sk_s_raw['incwage']
del tot_r_sk_s_raw['incwage_tot']

tot_r_sk_s_raw=tot_r_sk_s_raw[['sk','census_region_q','s','share_s_bysk']]

####merge on total labor endowment by region and skill type

tot_r_sk_lm=tot_r_sk[['census_region_q','sk','pop_skill_region']]

get_s_splits=tot_r_sk_s_raw.merge(tot_r_sk_lm,on=['census_region_q','sk'])

###multiply the percent of total labor in region by skill each household gets by total endow of labor type in that region
get_s_splits['s_endow_bysk']=get_s_splits['share_s_bysk']*get_s_splits['pop_skill_region']

tot_s_endow=get_s_splits.groupby(['census_region_q','s'],as_index=False).agg({'s_endow_bysk':sum})

tot_s_endow=tot_s_endow.rename(columns={'s_endow_bysk':'tot_s_endowment'})
tot_s_endow=tot_s_endow[['census_region_q','s','tot_s_endowment']]
get_s_splits=get_s_splits.merge(tot_s_endow,on=['census_region_q','s'])

#### get expenditure share by skill type 
get_s_splits['expenditure_share']=get_s_splits['s_endow_bysk']/get_s_splits['tot_s_endowment']
get_s_splits=get_s_splits[['sk','census_region_q','s','expenditure_share']]
####get actual household labor endowment from gams

prod_final=get_s_splits.merge(ld00_rh,on=['census_region_q','s'])
prod_final['pop_s_sk']=prod_final['expenditure_share']*prod_final['pop']



In [567]:
tot_r_sk_s_raw

,sk,census_region_q,s,share_s_bysk
0,skl,midwest,adm,0.011101
1,skl,midwest,agr,0.004775
2,skl,midwest,air,0.002476
3,skl,midwest,alt,0.000438
4,skl,midwest,amb,0.049545
...,...,...,...,...
563,unskl,west,wht,0.032439
564,unskl,west,wpd,0.005523
565,unskl,west,wrh,0.003342
566,unskl,west,wst,0.004767


In [564]:
tot_r_sk_cal=prod_final.groupby(['sk','census_region_q'],as_index=False).agg({'pop_s_sk':sum})

In [565]:
tot_r_sk_cal

,sk,census_region_q,pop_s_sk
0,skl,midwest,1051.271482
1,skl,northeast,1240.014635
2,skl,south,2009.613240
3,skl,west,1458.308258
4,unskl,midwest,719.797123
5,unskl,northeast,558.769025
6,unskl,south,1298.297892
7,unskl,west,879.682899


In [ ]:
####ok these datasets match on total pop, total pop by sector and total pop by household as defined in gams
####these datasets do not clear on skilled and unskilled labor demanded

##for consumer, we have the population by household for each census region
### 1) calculate percentage of total labor type in region demanded by household
### 2) calculate percentage of total labor type in region demanded by household
### 3) turn these into raw values based on gams percentages
### 4) calculate shares


In [527]:
tot_r_sk_raw

,census_region_q,sk,incwage_tot
0,midwest,skl,3.308015e+11
1,midwest,unskl,2.035075e+11
2,northeast,skl,4.219258e+11
3,northeast,unskl,1.699498e+11
4,south,skl,6.414238e+11
5,south,unskl,3.457185e+11
6,west,skl,4.577604e+11
7,west,unskl,2.226566e+11


In [525]:
prod_final.groupby(['census_region_q','s'],as_index=False).sum()

,census_region_q,s,expenditure_share,pop,pop_s_sk
0,midwest,adm,1.0,152.420907,76.210453
1,midwest,agr,1.0,18.010915,9.005457
2,midwest,air,1.0,20.068204,10.034102
3,midwest,alt,1.0,2.856363,1.428181
4,midwest,amb,1.0,221.875358,110.937679
...,...,...,...,...,...
279,west,wht,1.0,35.478958,17.739479
280,west,wpd,1.0,11.757643,5.878822
281,west,wrh,1.0,18.763410,9.381705
282,west,wst,1.0,16.083772,8.041886


In [526]:
ld00.groupby(['census_region_q','s'],as_index=False).sum()

,census_region_q,s,pop
0,midwest,adm,76.210453
1,midwest,agr,9.005457
2,midwest,air,10.034102
3,midwest,alt,1.428181
4,midwest,amb,110.937679
...,...,...,...
279,west,wht,17.739479
280,west,wpd,5.878822
281,west,wrh,9.381705
282,west,wst,8.041886


In [521]:
prod_final.head()

,sk,census_region_q,s,expenditure_share,pop,pop_s_sk
0,skl,midwest,adm,0.449746,76.210453,34.275326
1,unskl,midwest,adm,0.550254,76.210453,41.935127
2,skl,midwest,agr,0.329248,9.005457,2.965028
3,unskl,midwest,agr,0.670752,9.005457,6.040430
4,skl,midwest,air,0.640957,10.034102,6.431431


In [414]:
# #######get breakout of sector by skill type in wage data

# tot_r_sk_raw=all_breakouts_lm.groupby(['census_region_q','sk'],as_index=False).agg({'incwage':sum})
# tot_r_sk_raw=tot_r_sk_raw.rename(columns={'incwage':'incwage_tot'})
# tot_r_sk_h_raw=all_breakouts_lm.groupby(['census_region_q','sk','s'],as_index=False).agg({'incwage':sum})

# tot_r_sk_h_raw=tot_r_sk_h_raw.merge(tot_r_sk_raw,on=['census_region_q','sk'],how='inner')
# tot_r_sk_h_raw['share_s_bysk']=tot_r_sk_h_raw['incwage']/tot_r_sk_h_raw['incwage_tot']
# del tot_r_sk_h_raw['incwage']
# del tot_r_sk_h_raw['incwage_tot']

# ####merge share skill type by household onto previous table expressing share of labor type in each region

# prod_final=tot_r_sk_h_raw.merge(tot_r_sk,on=['census_region_q','sk'])
# prod_final['pop_region_skill_s']=prod_final['share_s_bysk']*prod_final['pop_skill_region']
# prod_final=prod_final[['census_region_q','s','sk','pop_region_skill_s','share_s_bysk']]

In [ ]:
##### now have:

1) endowment of each household by skill type
2) demand of each sector by skill type 
##### in this section I re-scale to create shares that are realtive to the total labor endowment by each skill type. I sum total demand by household and sector based on allocations based on percentage of total labor type, then create shares by these sums for each household and sector:
i.e. sector "adm" demands 1.11% of total skilled labor  and 2.2% of total unskilled labor. This sums to demands of 12.172 and 14.892 respectivly. Baase on this, their expendditure breakout is 12.172/12.172 + 14.892 and 14.892/12.172 + 14.892 for skilled and unskilled respectively.




##### now have:

1) datasets showing the percentage of skilled and unskilled labor totals in each region that each household is endowed with. i.e. in the midwest, hh1 makes up 3% of skilled labor,  and 13.5% of unskilled labor

2) datasets showing the percentage of skilled and unskilled labor totals in each region that each sector is demands.
   i.e. in the midwest, sector "adm" demands 1% of skilled labor and 2.2% of unskilled labor.
   
##### in this section I re-scale to create shares that are realtive to the total labor endowment by each skill type. I sum total demand by household and sector based on allocations based on percentage of total labor type, then create shares by these sums for each household and sector:
i.e. sector "adm" demands 1.11% of total skilled labor  and 2.2% of total unskilled labor. This sums to demands of 12.172 and 14.892 respectivly. Baase on this, their expendditure breakout is 12.172/12.172 + 14.892 and 14.892/12.172 + 14.892 for skilled and unskilled respectively.

#### Consumer side

In [415]:
####get total endowment by household
cons_sumbyhh=cons_final.groupby(['census_region_q','h'],as_index=False).agg({'pop_region_skill_hh':sum})
cons_sumbyhh=cons_sumbyhh.rename(columns={'pop_region_skill_hh':'pop_region_hh'})

###merge on to pop splits by skill
cons_final=cons_final.merge(cons_sumbyhh,on=['census_region_q','h'])

###recalculate expenditure shares based on percentage of total endowment allocated
cons_final['expenditure_shares']=cons_final['pop_region_skill_hh']/cons_final['pop_region_hh']
cons_final_lm=cons_final[['census_region_q','h','sk','pop_region_skill_hh','expenditure_shares']]

In [419]:
####get total endowment by sector
prod_sumbys=prod_final.groupby(['census_region_q','s'],as_index=False).agg({'pop_region_skill_s':sum})
prod_sumbys=prod_sumbys.rename(columns={'pop_region_skill_s':'pop_region_s'})

###merge on to pop splits by skill
prod_final=prod_final.merge(prod_sumbys,on=['census_region_q','s'])

###recalculate expenditure shares based on percentage of total demand allocated
prod_final['expenditure_shares']=prod_final['pop_region_skill_s']/prod_final['pop_region_s']
prod_final_lm=prod_final[['census_region_q','s','sk','pop_region_skill_s','expenditure_shares']]

In [463]:
cons_final_lm.groupby(['census_region_q','h'],as_index=False).agg({'pop_region_skill_hh':sum})['pop_region_skill_hh'].sum()




9215.754554875697

In [462]:
le00.groupby(['h','census_region_r'],as_index=False).sum()['pop'].sum()

9215.754554875695

In [450]:
ld00.groupby(['sk','census_region_q'],as_index=False).sum().head()

,sk,census_region_q,pop
0,skl,midwest,1313.697242
1,skl,northeast,1466.218037
2,skl,south,2445.644111
3,skl,west,1789.787203
4,unskl,midwest,457.371363


##### now merge on all combinations for consumer and producer side

In [438]:
####consumers
####group labor endowment dataset across skill to get "pop" by r,q,h

le00_sum=le00.groupby(['r','q','h','census_region_q'],as_index=False).sum()

####merge this dataset with consumer endowment shares by skill

cons_breakouts=le00_sum.merge(cons_final_lm,on=['census_region_q','h'])

cons_breakouts['final_pop']=cons_breakouts['pop']*cons_breakouts['expenditure_shares']
cons_breakouts_final=cons_breakouts[['r','q','sk','h','census_region_q','expenditure_shares','final_pop']]


In [439]:
####consumers
####group labor endowment dataset across skill to get "pop" by r,q,h

ld00_sum=ld00.groupby(['q','s','census_region_q'],as_index=False).sum()

####merge this dataset with producer expenditure shares by skill

prod_breakouts=ld00_sum.merge(prod_final_lm,on=['census_region_q','s'])

prod_breakouts['final_pop']=prod_breakouts['pop']*prod_breakouts['expenditure_shares']
prod_breakouts_final=prod_breakouts[['q','s','sk','census_region_q','expenditure_shares','final_pop']]


In [443]:
cons_breakouts_final.groupby(['census_region_q','sk'],as_index=False).agg({'final_pop':sum})

,census_region_q,sk,final_pop
0,midwest,skl,1323.321559
1,midwest,unskl,447.747046
2,northeast,skl,1458.065260
3,northeast,unskl,340.718401
4,south,skl,2479.154734
5,south,unskl,828.756398
6,west,skl,1808.007940
7,west,unskl,529.983217


In [444]:
prod_breakouts_final.groupby(['census_region_q','sk'],as_index=False).agg({'final_pop':sum})

,census_region_q,sk,final_pop
0,midwest,skl,1051.271482
1,midwest,unskl,719.797123
2,northeast,skl,1240.014635
3,northeast,unskl,558.769025
4,south,skl,2009.613240
5,south,unskl,1298.297892
6,west,skl,1458.308258
7,west,unskl,879.682899


In [ ]:
#share of total regional skilled labor each sector demands

In [ ]:
#share of total regional skilled labor each household endows
#share of household endowment by labor type

In [ ]:
household 1 makes up 10% of total labor endowment, of that 10%, 40% is skilled, 60% is unskilled

In [ ]:
sector j makes up 10% of total labor demand, of that 10%, 40% is skilled, 60% is unskilled

In [347]:
prod_final.head()

,census_region_q,s,sk,pop_region_skill_s,census_share,pop_skill_region
0,midwest,adm,skl,12.171775,0.011101,1096.50432
1,midwest,agr,skl,5.235977,0.004775,1096.50432
2,midwest,air,skl,2.714881,0.002476,1096.50432
3,midwest,alt,skl,0.480763,0.000438,1096.50432
4,midwest,amb,skl,54.326132,0.049545,1096.50432


In [351]:
le00_s['pop'].sum()

9215.754554875695

In [352]:
ca_rqhsk['pop'].sum()

18431.50910975139

In [339]:
#### merge on final dataset prod

ld00_s=ld00.groupby(['q','s','census_region_q'],as_index=False).sum()

all_breakouts_aug_prod=all_breakouts_lm.merge(ld00, on=['q','s','sk','census_region_q'],how='right')

del all_breakouts_aug_prod['pop']

all_breakouts_aug_prod2=all_breakouts_aug_prod.merge(ld00_s, on=['q','s','census_region_q'],how='outer',indicator=True)



# merge census averages onto original merged dataframe
ca_qssk=all_breakouts_aug_prod2.merge(prod_final,on=['census_region_q','s','sk'],how='inner')
ca_qssk=ca_qssk.groupby(['q','s','sk','census_region_q'],as_index=False).agg({'census_share':max,'pop':max})
ca_qssk['pop_final']=ca_qssk['census_share']*ca_qssk['pop']


In [341]:
ca_qssk.groupby(['sk','census_region_q'],as_index=False).agg({'pop_final':sum})

,sk,census_region_q,pop_final
0,skl,midwest,47.240609
1,skl,northeast,58.769589
2,skl,south,101.204907
3,skl,west,69.300457
4,unskl,midwest,39.757898
5,unskl,northeast,42.916647
6,unskl,south,89.691355
7,unskl,west,61.578128


In [342]:
ca_rqhsk.groupby(['sk','census_region_q'],as_index=False).agg({'pop_final':sum})

,sk,census_region_q,pop_final
0,skl,midwest,483.863429
1,skl,northeast,592.907630
2,skl,south,965.360754
3,skl,west,738.755030
4,unskl,midwest,273.469143
5,unskl,northeast,297.526456
6,unskl,south,551.807282
7,unskl,west,401.204136


In [ ]:
#get percent of all labor in region that a household comprises
#i.e. midwest hh1 is 10%
#get percent breakout that each household is of skilled and unskilled
# 

In [ ]:
#of skilled labor in the midwest, hh1 is 10%
# hh1 units are 40% skilled 60% unskilled

In [357]:
prod_final

,census_region_q,s,sk,pop_region_skill_s,census_share,pop_skill_region
0,midwest,adm,skl,12.171775,0.011101,1096.504320
1,midwest,agr,skl,5.235977,0.004775,1096.504320
2,midwest,air,skl,2.714881,0.002476,1096.504320
3,midwest,alt,skl,0.480763,0.000438,1096.504320
4,midwest,amb,skl,54.326132,0.049545,1096.504320
...,...,...,...,...,...,...
563,west,wht,unskl,24.818046,0.032439,765.073702
564,west,wpd,unskl,4.225229,0.005523,765.073702
565,west,wrh,unskl,2.556938,0.003342,765.073702
566,west,wst,unskl,3.647106,0.004767,765.073702


In [ ]:
#of the skilled expenditures demanded in the west, adm gets 0.01 pct
#of the unskilled labor expenditures in the west, adm gets 0.004 pct

#adm gets a total of (0.01*1789.7872029262608)+(0.004*1789.7872029262608) = 25.057 units

#but data says he gets 51

#rescale total expenditures in west to be this large .014x=51, =3642.8571428571427



In [ ]:
#in gams table, allocate 1% of total skilled expenditures to adm skilled
#in gams table, allocate 4% of total unskilled expenditures to adm unskilled

#get that total, get that breakout

# recalculate breakout on actual labor endowment

In [365]:
51/.014

3642.8571428571427

In [366]:
ld00[(ld00['census_region_q']=='west')& (ld00['sk']=='skl')]

,q,s,sk,pop,census_region_q
988,AK,ppd,skl,0.000162,west
990,AK,res,skl,0.523755,west
992,AK,com,skl,0.071469,west
994,AK,amb,skl,0.910772,west
996,AK,fbp,skl,0.469571,west
...,...,...,...,...,...
7192,CO,air,skl,1.410582,west
7194,CO,mmf,skl,0.859116,west
7196,CO,otr,skl,0.793506,west
7198,CO,min,skl,0.376623,west


In [363]:
(0.01*1789.7872029262608)+(0.004*1789.7872029262608)

25.057020840967652

In [364]:
prod_final

,census_region_q,s,sk,pop_region_skill_s,census_share,pop_skill_region
0,midwest,adm,skl,12.171775,0.011101,1096.504320
1,midwest,agr,skl,5.235977,0.004775,1096.504320
2,midwest,air,skl,2.714881,0.002476,1096.504320
3,midwest,alt,skl,0.480763,0.000438,1096.504320
4,midwest,amb,skl,54.326132,0.049545,1096.504320
...,...,...,...,...,...,...
563,west,wht,unskl,24.818046,0.032439,765.073702
564,west,wpd,unskl,4.225229,0.005523,765.073702
565,west,wrh,unskl,2.556938,0.003342,765.073702
566,west,wst,unskl,3.647106,0.004767,765.073702


In [358]:
ca_qssk[(ca_qssk['census_region_q']=='west')& (ca_qssk['sk']=='skl')]

,q,s,sk,census_region_q,census_share,pop,new_pop
0,AK,adm,skl,west,0.444168,0.511427,0.227159
2,AK,agr,skl,west,0.280780,0.013264,0.003724
4,AK,air,skl,west,0.622915,0.554755,0.345566
6,AK,alt,skl,west,0.475725,0.001177,0.000560
8,AK,amb,skl,west,0.785563,1.479738,1.162428
...,...,...,...,...,...,...,...
7192,WY,wht,skl,west,0.520532,0.100860,0.052501
7194,WY,wpd,skl,west,0.431722,0.031142,0.013445
7196,WY,wrh,skl,west,0.216786,0.068964,0.014950
7198,WY,wst,skl,west,0.281717,0.044173,0.012444


In [337]:
cons_final.head()

,census_region_q,h,sk,pop_region_skill_hh,census_share
0,midwest,hh1,skl,33.353419,0.030418
1,midwest,hh2,skl,165.594920,0.151021
2,midwest,hh3,skl,238.496815,0.217506
3,midwest,hh4,skl,370.736584,0.338108
4,midwest,hh5,skl,288.322582,0.262947


In [333]:
prod_final.groupby(['census_region_q','sk'],as_index=False).sum()

,census_region_q,sk,pop_region_skill_s,census_share
0,midwest,skl,1096.504320,1.0
1,midwest,unskl,674.564285,1.0
2,northeast,skl,1282.285148,1.0
3,northeast,unskl,516.498513,1.0
4,south,skl,2149.409358,1.0
5,south,unskl,1158.501774,1.0
6,west,skl,1572.917455,1.0
7,west,unskl,765.073702,1.0


In [332]:
prod_final

,census_region_q,s,sk,pop_region_skill_s,census_share
0,midwest,adm,skl,12.171775,0.011101
1,midwest,agr,skl,5.235977,0.004775
2,midwest,air,skl,2.714881,0.002476
3,midwest,alt,skl,0.480763,0.000438
4,midwest,amb,skl,54.326132,0.049545
...,...,...,...,...,...
563,west,wht,unskl,24.818046,0.032439
564,west,wpd,unskl,4.225229,0.005523
565,west,wrh,unskl,2.556938,0.003342
566,west,wst,unskl,3.647106,0.004767


In [326]:
cons_final.groupby(['census_region_q','sk'],as_index=False).sum()

,census_region_q,sk,pop_region_skill_hh
0,midwest,skl,1096.504320
1,midwest,unskl,674.564285
2,northeast,skl,1282.285148
3,northeast,unskl,516.498513
4,south,skl,2149.409358
5,south,unskl,1158.501774
6,west,skl,1572.917455
7,west,unskl,765.073702


#### Assign census shares to all breakouts

In [353]:
### for r,q,h,sk breakouts get skill shares of expenditures by census region working in

census_avg_tot0=all_breakouts_lm.groupby(['census_region_q','h'],as_index=False).agg({'incwage':sum})
census_avg_tot0=census_avg_tot0.rename(columns={'incwage':'incwage_tot'})
census_avg0=all_breakouts_lm.groupby(['census_region_q','h','sk'],as_index=False).agg({'incwage':sum})

census_avg0=census_avg0.merge(census_avg_tot0,on=['census_region_q','h'],how='inner')
census_avg0['census_share']=census_avg0['incwage']/census_avg0['incwage_tot']
del census_avg0['incwage']
del census_avg0['incwage_tot']

le00_s=le00.groupby(['r','q','h','census_region_q'],as_index=False).sum()

all_breakouts_aug_cons=all_breakouts_lm.merge(le00, on=['r','q','h','sk','census_region_q'],how='right')
del all_breakouts_aug_cons['pop']
all_breakouts_aug_cons2=all_breakouts_aug_cons.merge(le00_s, on=['r','q','h','census_region_q'],how='outer',indicator=True)


# merge census averages onto original merged dataframe
ca_rqhsk=all_breakouts_aug_cons2.merge(census_avg0,on=['census_region_q','h','sk'],how='inner')
ca_rqhsk=ca_rqhsk.groupby(['r','q','h','sk','census_region_q'],as_index=False).agg({'census_share':max,'pop':max})
ca_rqhsk['new_pop']=ca_rqhsk['census_share']*ca_rqhsk['pop']


###########################
###########################


### for r,s,sk breakouts get skill shares of expenditures by census region industry is in 

census_avg_tot1=all_breakouts_lm.groupby(['census_region_q','s'],as_index=False).agg({'incwage':sum})
census_avg_tot1=census_avg_tot1.rename(columns={'incwage':'incwage_tot'})
census_avg1=all_breakouts_lm.groupby(['census_region_q','s','sk'],as_index=False).agg({'incwage':sum})


census_avg1=census_avg1.merge(census_avg_tot1,on=['census_region_q','s'],how='inner')
census_avg1['census_share']=census_avg1['incwage']/census_avg1['incwage_tot']
del census_avg1['incwage']
del census_avg1['incwage_tot']

ld00_s=ld00.groupby(['q','s','census_region_q'],as_index=False).sum()

all_breakouts_aug_prod=all_breakouts_lm.merge(ld00, on=['q','s','sk','census_region_q'],how='right')
del all_breakouts_aug_prod['pop']
all_breakouts_aug_prod2=all_breakouts_aug_prod.merge(ld00_s, on=['q','s','census_region_q'],how='outer',indicator=True)



# merge census averages onto original merged dataframe
ca_qssk=all_breakouts_aug_prod2.merge(census_avg1,on=['census_region_q','s','sk'],how='inner')
ca_qssk=ca_qssk.groupby(['q','s','sk','census_region_q'],as_index=False).agg({'census_share':max,'pop':max})
ca_qssk['new_pop']=ca_qssk['census_share']*ca_qssk['pop']


In [354]:
ca_qssk['new_pop'].sum()

9215.754554865573

In [356]:
ca_rqhsk['new_pop'].sum()

9215.754554875697

##### ak, hi, co, wa, ca, oil sector all demand census level share of oil demand for unskilled labor

##### ak, hi, co, wa, ca, hh1 all have census level share of hh1 unskilled labor 

In [242]:
tot=ca_rqhsk[(ca_rqhsk['q'].isin(west)) & (ca_rqhsk['h'].isin(['hh1']))]['new_pop'].sum()

unskl=ca_rqhsk[(ca_rqhsk['q'].isin(west)) & (ca_rqhsk['h'].isin(['hh1'])) & (ca_rqhsk['sk']=='unskl')]['new_pop'].sum()

In [243]:
ca_rqhsk[(ca_rqhsk['q'].isin(west)) & (ca_rqhsk['h'].isin(['hh1']))]

,r,q,h,sk,census_region_q,census_share,pop,new_pop
0,AK,AK,hh1,skl,west,0.263268,0.181200,0.047704
1,AK,AK,hh1,unskl,west,0.736732,0.181200,0.133496
10,AK,AZ,hh1,skl,west,0.263268,0.007569,0.001993
11,AK,AZ,hh1,unskl,west,0.736732,0.007569,0.005576
20,AK,CA,hh1,skl,west,0.263268,0.007592,0.001999
...,...,...,...,...,...,...,...,...
6801,WY,UT,hh1,unskl,west,0.736732,0.008743,0.006441
6810,WY,WA,hh1,skl,west,0.263268,0.008838,0.002327
6811,WY,WA,hh1,unskl,west,0.736732,0.008838,0.006511
6820,WY,WY,hh1,skl,west,0.263268,0.210360,0.055381


In [276]:
ca_rqhsk[(ca_rqhsk['q'].isin(south))&(ca_rqhsk['r'].isin(south))  ].to_excel('/Users/hannahkamen/Downloads/h_test.xlsx')


In [278]:
ca_qssk[(ca_qssk['q'].isin(south))  ].to_excel('/Users/hannahkamen/Downloads/s_test.xlsx')




In [266]:
ca_rqhsk[(ca_rqhsk['q'].isin(west)) &  (ca_rqhsk['sk'].isin(['skl']))]['new_pop'].sum()

1808.007939931418

In [254]:
tot=ca_qssk[(ca_qssk['q'].isin(west)) & (ca_qssk['s'].isin(['adm']))]['new_pop'].sum()

unskl=ca_qssk[(ca_qssk['q'].isin(west)) & (ca_qssk['s'].isin(['adm'])) & (ca_qssk['sk']=='unskl')]['new_pop'].sum()


ca_qssk[(ca_qssk['q'].isin(west)) & (ca_qssk['s'].isin(['adm']))]

,q,s,sk,census_region_q,census_share,pop,new_pop
0,AK,adm,skl,west,0.444168,0.511427,0.227159
1,AK,adm,unskl,west,0.555832,0.511427,0.284267
424,AZ,adm,skl,west,0.444168,10.492343,4.660361
425,AZ,adm,unskl,west,0.555832,10.492343,5.831982
566,CA,adm,skl,west,0.444168,53.946409,23.961260
567,CA,adm,unskl,west,0.555832,53.946409,29.985149
708,CO,adm,skl,west,0.444168,6.992400,3.105799
709,CO,adm,unskl,west,0.555832,6.992400,3.886601
1544,HI,adm,skl,west,0.444168,1.971856,0.875835
1545,HI,adm,unskl,west,0.555832,1.971856,1.096021


In [259]:
0.449746+0.550254

1.0

In [258]:
ca_qssk.groupby(['s','sk','census_region_q'],as_index=False).agg({'census_share':max})

,s,sk,census_region_q,census_share
0,adm,skl,midwest,0.449746
1,adm,skl,northeast,0.516942
2,adm,skl,south,0.451408
3,adm,skl,west,0.444168
4,adm,unskl,midwest,0.550254
...,...,...,...,...
563,wtt,skl,west,0.506243
564,wtt,unskl,midwest,0.688010
565,wtt,unskl,northeast,0.360629
566,wtt,unskl,south,0.523297


In [255]:
unskl/tot

0.5558321592472192

In [282]:
ca_rqhsk.groupby(['q','census_region_q'],as_index=False).sum()[['q','new_pop']]

,q,new_pop
0,AK,35.793842
1,AL,101.715888
2,AR,56.261682
3,AZ,158.148475
4,CA,1239.295208
5,CO,180.990251
6,CT,116.950938
7,DC,93.350221
8,DE,30.429920
9,FL,478.104818


In [281]:
ca_qssk.groupby(['q','census_region_q'],as_index=False).sum()[['q','new_pop','census_region_q']]

,q,new_pop,census_region_q
0,AK,35.793842,west
1,AL,101.715888,south
2,AR,56.261682,south
3,AZ,158.148475,west
4,CA,1239.295208,west
5,CO,180.990251,west
6,CT,116.950938,northeast
7,DC,93.350221,south
8,DE,30.429920,south
9,FL,478.104818,south


In [ ]:
each of r,q,h

In [22]:
##outer merge CPS data with gams table with indicator
merge_gams=gams_rqhs_lm1.merge(le00,on=['r','q','h','sk'],how='outer',indicator=True)
###get census region and assign average skilled and unskilled breakouts by census region for missing combinations
merge_gams['census_region']=np.where(merge_gams['r'].isin(midwest),'midwest','')
merge_gams['census_region']=np.where(merge_gams['r'].isin(south),'south',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(west),'west',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(northeast),'northeast',merge_gams['census_region'])

##get skilled unskilled breakouts by industry and census region

census_avg_tot0=merge_gams.groupby(['census_region','h'],as_index=False).agg({'incwage':sum})
census_avg_tot0=census_avg_tot0.rename(columns={'incwage':'incwage_tot'})
census_avg0=merge_gams.groupby(['census_region','h','sk'],as_index=False).agg({'incwage':sum})

census_avg0=census_avg0.merge(census_avg_tot0,on=['census_region','h'],how='inner')
census_avg0['census_share']=census_avg0['incwage']/census_avg0['incwage_tot']
del census_avg0['incwage']
del census_avg0['incwage_tot']

# merge census averages onto original merged dataframe
merge_gams_cs=merge_gams.merge(census_avg0,on=['census_region','h','sk'],how='inner')

##keep only inner values and values missing from CPS

final0=merge_gams_cs[merge_gams_cs['_merge'].isin(['right_only','both'])]

##get skilled unskilled breakouts by industry (ignore missing values for now)

final_tot0=final0.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})
final_tot0=final_tot0.rename(columns={'incwage':'incwage_tot'})


final_shrs0=final0.merge(final_tot0,on=['r','q','h'],how='inner')
final_shrs0['skill_shr']=final_shrs0['incwage']/final_shrs0['incwage_tot']
del final_shrs0['incwage']
del final_shrs0['incwage_tot']

#create tag to define missing combinations in final dataset

final_shrs0['combo']=final_shrs0['r']+"_"+final_shrs0['q']+"_"+final_shrs0['h']+"_"+final_shrs0['sk']

## get list of missing combionations using _merge indicator var

missing_combos0=final_shrs0[final_shrs0['_merge']=='right_only']['r']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['q']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['h']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['sk']

# ###if region sector combination is not missing, use census region values, otherwise keep original share value
# final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(missing_combos0),final_shrs0['census_share'],final_shrs0['skill_shr'])
# final_shrs0['skill_shr']=np.where(final_shrs0['combo']=='ME_NJ_hh1',final_shrs0['census_share'],final_shrs0['skill_shr'])




# ###limit dataframe

# final_shrs0=final_shrs0[['r','q','s','h','sk','skill_shr','combo','pop','census_share']]

# #chksum to see that all proportions add to 1
# check_sum0=final_shrs0.groupby(['r','q','s','h'],as_index=False).agg({'skill_shr':sum})
# check_sum0=check_sum0[check_sum0['skill_shr']!=1]


# #chksum to see that no zeros exist
# check_sum10=final_shrs0.groupby(['r','q','s','h','sk'],as_index=False).agg({'skill_shr':sum})
# check_sum10=check_sum10[check_sum10['skill_shr']==0]

# zeros=check_sum10['r']+"_"+check_sum10['q']+"_"+check_sum10['s']+"_"+check_sum10['h']


# #replace 0s with census values

# final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(zeros),final_shrs0['census_share'],final_shrs0['skill_shr'])



# #check zeros agains
# check_sum10=final_shrs0.groupby(['r','q','s','h','sk'],as_index=False).agg({'skill_shr':sum})
# check_sum10=check_sum10[check_sum10['skill_shr']==0]

# final_shrs0_f=final_shrs0[['r','q','s','h','sk','skill_shr']]



In [23]:
missing_combos0

0         IA_IA_hh1_skl
1       IA_IA_hh1_unskl
2         IA_OH_hh1_skl
3       IA_OH_hh1_unskl
4         IA_WI_hh1_skl
             ...       
6825    CO_MT_hh5_unskl
6826      CO_WY_hh5_skl
6827    CO_WY_hh5_unskl
6828      CO_CO_hh5_skl
6829    CO_CO_hh5_unskl
Length: 6830, dtype: object

In [ ]:
final_shrs0[final_shrs0['census_share']==0]

In [ ]:
zeros

In [ ]:
check_sum10

In [ ]:
missing_combos0

In [ ]:
check_sum0

In [ ]:
final_shrs0[final_shrs0['combo'].isin(missing_combos0)]['pop'].sum()

In [ ]:
469.6733563736173/9215

In [ ]:
len(final_shrs0)

In [ ]:
missing_combos0